In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 引入 parse 模組
import urllib.parse 

# 整理 csv 使用的工具
import csv

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re

'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
#options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

#視窗大小設定 (寬，高)
# driver.set_window_size(1200, 960)

driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化


'''
自訂變數
'''
# 放置爬取的資料
house_data = []
house_detail_data = []

# 放置 所有超連結
URLs_data = [] 

#全域變數
page = 0

def openweb():
    #前往頁面
    driver.get("https://newhouse.591.com.tw/housing-list.html?rid=1&page=1");
    
    # 強制等待
    sleep(5)



# 分析頁面元素資訊
def parse():
    global page
    
    sleep(2)
    # 取得主要元素的集合
    house_list = driver.find_elements(By.CSS_SELECTOR, '#vue-list > ul > span > li')
    # 逐一檢視元素
    for index, house_urls in enumerate(house_list):
        # 取得連結
        a = house_urls.find_element(By.CSS_SELECTOR, ' a ')
        house_url = a.get_attribute('href')
        
        URLs_data.append({
            "URLs":house_url
         })

    #設定爬取的頁數
    if page+1 < 2:

        sleep(0.3)
        if len(driver.find_elements(By.CSS_SELECTOR, 'a.pageNext') ) > 0:
            sleep(0.3)
            driver.find_element(By.CSS_SELECTOR, 'a.pageNext').click()
            page+=1
            parse()
            
    else:
        visit_URLs()
    
        
        
def visit_URLs():
    i=0
    for every_URLs in URLs_data:
        #讀取連結
        house_URL = every_URLs['URLs']
        #前往每個連結
        print(str(house_URL))
        driver.get(str(house_URL))
        sleep(1)
        
        get_data()
        
        house_data[i].extend(get_detail_data())
        i+=1

def get_data():

    house_infos = driver.find_elements(By.CSS_SELECTOR, 'div.detail_info > div.detail_info_list')
    for house_info in house_infos:
        
        single_price= house_info.find_element_by_class_name('total_price.stonefont').text
        print("單坪:"+single_price+"萬")
        
        #取車位字元位子
        carpark_price_loc = house_info.find_element_by_class_name('detail_price.stonefont').text.find('車位')  
        
        total_price = house_info.find_element_by_class_name('detail_price.stonefont').text[4:carpark_price_loc]
        print("總價:"+total_price)
        carpark_price = house_info.find_element_by_class_name('detail_price.stonefont').text[carpark_price_loc+4:]
        print("車位:"+carpark_price)
        
        house_value = house_info.find_element_by_class_name('pattern-right').text
        print("格局:"+house_value)
        house_addr = house_info.find_element_by_class_name('address.stonefont').text[5:-4]
        print("地址:"+house_addr)
        
            
        house_data.append([
            single_price,
            total_price,
            carpark_price,
            house_value,
            house_addr
        ])       
        
        
def get_detail_data():    
    #詳細資訊
    house_detail_data = ['Null','Null','Null','Null','Null','Null','Null','Null','Null','Null','Null','Null','Null','Null']
    sleep(0.5)
    
    for num in range(14):
        
        if num == 0 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[7:]
        elif num == 1 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 2 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[9:]
        elif num == 3 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 4 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 5 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 6 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 7 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 8 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[6:]
        elif num == 9 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 10 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 11 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 12 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[4:]
        elif num == 13 :
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text[6:]
        else:
            house_detail_data[num] = driver.find_elements(By.CSS_SELECTOR,'div.build_plan.list_title > ul > li')[num].text

    return house_detail_data
    

                 
openweb()
parse()

#不指定爬蟲頁數下面再取消註解
# visit_URLs()


In [ ]:
def createcsv():
    
    #print(house_detail_data)
    house_data.insert(0,['單坪','總價','車位','格局','地址','公設比','棟戶規劃','建蔽率','樓層規劃','車位規劃','管理費用','車位配比','結構工程','基地面積','用途規劃','交屋屋況','土地分區','座向規劃','建材說明'])
    print(house_data)
    csvfile=open('housedata.csv','w', newline='',encoding="utf-8")
    obj = csv.writer(csvfile)
    for row in house_data:
        obj.writerow(row)
    csvfile.close()          

createcsv()             
    